<a href="https://colab.research.google.com/github/PritiRangnekar/CS221FinalProjectEllo/blob/main/BARTModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import torch
from torch.utils.data import Dataset
!pip install transformers
import transformers
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    BartTokenizer,
)
!pip install datasets
import datasets
from datasets import load_dataset
from transformers import BartTokenizer, BartForCausalLM
# Use Google Collab's GPU
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    print('No GPU available')
  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.4 MB/s 
     |████████████████████████████████| 596 kB 54.4 MB/s 
     |████████████████████████████████| 6.6 MB 37.5 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 7.1 MB/s 
     |████████████████████████████████| 86 kB 7.3 MB/s 
     |████████████████████████████████| 212 kB 67.4 MB/s 
     |████████████████████████████████| 140 kB 71.0 MB/s 
     |████████████████████████████████| 1.1 MB 59.1 MB/s 
     |████████████████████████████████| 127 kB 60.2 MB/s 
     |████████████████████████████████| 271 kB 62.6 MB/s 
     |███████████████

In [2]:
# Set our random seed for running torch
torch.manual_seed(42)

# Download the tokenizer for Bart from the huggingface library 
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
# Download the pre-trained Bart model from huggingface and transfer it to the GPU
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base", add_cross_attention=False).cuda()
#model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [3]:
# Function for pre-processing (tokenizing, separating) our dataset
def preprocess_function(examples):
    max_length = 512
    model_inputs = tokenizer(examples["Text"], max_length=512, truncation=True,padding=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Commentary"], max_length=512, truncation=True, padding=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
# Create our tokenized train and val dataset to train with
dataset = load_dataset('csv', data_files='CONTEXT_b1Train')
tokenized_dataset = dataset.map(preprocess_function)

val_data = load_dataset('csv', data_files='CONTEXT_b1Val')
tokenized_val_dataset = val_data.map(preprocess_function)

Using custom data configuration default-6b2b44d1eb60094b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6b2b44d1eb60094b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Parameter 'function'=<function preprocess_function at 0x7fb18ceb3a70> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


...

Using custom data configuration default-229e5536088397e2


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-229e5536088397e2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?ex/s]

In [7]:
# define our data collator:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# define our training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,
    weight_decay=0.01,
    num_train_epochs=10,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_val_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 672
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 3360
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: ISBN, Page, Commentary, Text. If ISBN, Page, Commentary, Text are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss
1,No log,2.332899
2,2.642100,2.348574
3,1.693500,2.451738
4,1.693500,2.704855
5,1.011900,2.946795
6,0.649800,3.136827
7,0.649800,3.283474
8,0.353700,3.325804
9,0.224100,3.486229
10,0.224100,3.487413


***** Running Evaluation *****
  Num examples = 84
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: ISBN, Page, Commentary, Text. If ISBN, Page, Commentary, Text are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 84
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: ISBN, Page, Commentary, Text. If ISBN, Page, Commentary, Text are not expected by

TrainOutput(global_step=3360, training_loss=0.995754337310791, metrics={'train_runtime': 418.0625, 'train_samples_per_second': 16.074, 'train_steps_per_second': 8.037, 'total_flos': 333716385300480.0, 'train_loss': 0.995754337310791, 'epoch': 10.0})

In [ ]:
# function for reading our test and val into a dataframe to be used for the output csv 
def read_data(file_path): 
  df = pd.read_csv(file_path, header=None)
  df.columns = ['Text', 'Commentary', 'ISBN', 'Page']
  x, y, isbn, page = df['Text'].tolist(), df['Commentary'].tolist(), df['ISBN'].tolist(), df['Page'].tolist()
  x.pop(0)
  y.pop(0)
  isbn.pop(0)
  page.pop(0)
  return x, y, isbn, page


# function for just generating our output commentary given input page-text 
def generate_summary(x_test):
    inputs = tokenizer(
        x_test,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

# read our test and val into a dataframe
test_file_path = 'batchOneTest'
eval_file_path = 'batchOneVal'
x_eval, y_eval, isbn_eval, page_eval = read_data('batchOneVal')
x_test, y_test, isbn_test, page_test = read_dat('batchOneTest')

# generate commentaries, and use them as the "commentary" to export into csv
test_commentaries = generate_summary(x_test)[1]
eval_commentaries = generate_summary(x_eval)[1]

eval_csv = pd.DataFrame({'Text': x_eval, 'Commentary': eval_commentaries, 'ISBN': isbn_eval, 'Page': page_eval})
eval_csv.to_csv('BARTCommentaries_batchOneVal2.csv')

test_csv = pd.DataFrame({'Text': x_test, 'Commentary': test_commentaries, 'ISBN': isbn_test, 'Page': page_test})
test_csv.to_csv('BARTCommentaries_batchOneTest2.csv')